<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#New-columns-added" data-toc-modified-id="New-columns-added-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>New columns added</a></span></li><li><span><a href="#Load-cleaned-Teranet-dataset-with-da_id-column" data-toc-modified-id="Load-cleaned-Teranet-dataset-with-da_id-column-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load cleaned Teranet dataset with <code>da_id</code> column</a></span><ul class="toc-item"><li><span><a href="#Drop-all-records-with-missing-(or-&lt;$30)-consideration_amt" data-toc-modified-id="Drop-all-records-with-missing-(or-<$30)-consideration_amt-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Drop all records with missing (or &lt;$30) <code>consideration_amt</code></a></span></li><li><span><a href="#Unique-records-by-pin" data-toc-modified-id="Unique-records-by-pin-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Unique records by <code>pin</code></a></span></li><li><span><a href="#Unique-records-by-xy" data-toc-modified-id="Unique-records-by-xy-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Unique records by <code>xy</code></a></span></li></ul></li><li><span><a href="#Add-column-total_sales" data-toc-modified-id="Add-column-total_sales-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Add column <code>total_sales</code></a></span></li><li><span><a href="#Add-column-prev_sales" data-toc-modified-id="Add-column-prev_sales-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Add column <code>prev_sales</code></a></span></li><li><span><a href="#Add-columns-price_cum_sum-and-price_pct_change" data-toc-modified-id="Add-columns-price_cum_sum-and-price_pct_change-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Add columns <code>price_cum_sum</code> and <code>price_pct_change</code></a></span></li><li><span><a href="#Add-column-price_da_pct_change" data-toc-modified-id="Add-column-price_da_pct_change-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Add column <code>price_da_pct_change</code></a></span></li><li><span><a href="#Add-columns-years_since_last_sale" data-toc-modified-id="Add-columns-years_since_last_sale-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Add columns <code>years_since_last_sale</code></a></span></li><li><span><a href="#Add-columns-da_days_since_last_sale-and-da_years_since_last_sale" data-toc-modified-id="Add-columns-da_days_since_last_sale-and-da_years_since_last_sale-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Add columns <code>da_days_since_last_sale</code> and <code>da_years_since_last_sale</code></a></span></li><li><span><a href="#Add-columns-sale_next_6m/1y/3y-per-pin-and-xy" data-toc-modified-id="Add-columns-sale_next_6m/1y/3y-per-pin-and-xy-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Add columns <code>sale_next_6m/1y/3y</code> per <code>pin</code> and <code>xy</code></a></span></li><li><span><a href="#Save-results-to-a-new-.csv-file" data-toc-modified-id="Save-results-to-a-new-.csv-file-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Save results to a new .csv file</a></span></li></ul></div>

# Feature extraction: Teranet records
This notebook presents the process of feature extraction from Teranet records that previously were:
* cleaned and filtered for duplicates
    * `consideration_amt` < $30 were reset to NaN (Not a Number, missing values)
    * records matching on all columns have been removed (83'798 records)
    * records matching on all columns excluding `pin` have been removed (729'182 records)
    * **813'138 duplicate entries** removed in total from original Teranet dataset 
    * 8'226'103 unique records remain after duplicates have been removed
    * see notebook `data_cleaning/Teranet_data_cleaning.ipynb` for details
* filtered to include only records from GTHA 
    * filtering performed via a spatial join
    * `xy` coordinates of Teranet records joined (how='inner', op='within') with DA geometry for GTHA 
    * DA geometry provided by York Municipal Government (accessed via Esri Open Data portal)
    * 6,062,853 records have `xy` coordinates within GTHA boundary
    * see notebook `data_cleaning/Teranet_GTHA_DA_spatial_join.ipynb` for details

## New columns added
* new columns added (available on transaction level):
    * `da_id`, `da_city`, `da_median_tot_inc`: were added during the spatial join with DA data
    * `xy`: `x` and `y` concatenated together (used for grouping by coordinate pairs)
    * `pin/xy_total_sales`: total records for this `pin`/`xy`
    * `pin/xy_prev_sales`: previous records from this `pin`/`xy` (not counting current transaction)
    * `pin/xy_price_cum_sum`: cumulative price of all records to date from this `pin`/`xy`
    * `pin/xy_price_pct_change`: price percentage change compared to previous record from this `pin`/`xy`
    * `price_da_pct_change`: price percentage change compared to previous record from this DA (by `da_id`)
    * `pin/xy_years_since_last_sale`: years since last sale from this `pin`/`xy`
    * `da_days_since_last_sale`, `da_years_since_last_sale`: days or years since last sale from this DA (by `da_id`)
    * `sale_next_6m/1y/3y`: "looks into the future" to see whether there is another transaction from this `pin`/`xy` within the given time horizon (6 months, 1 year, 3 years)

In [1]:
import pandas as pd
import time
import os

In [2]:
# Set some parameters to apply to all plots. These can be overridden
# in each plot if desired
import matplotlib
# Plot size to 14" x 7"
# matplotlib.rc('figure', figsize = (14, 7))
# Font size to 14
matplotlib.rc('font', size = 14)
# Do not display top and right frame lines
matplotlib.rc('axes.spines', top = False, right = False)
# Remove grid lines
matplotlib.rc('axes', grid = False)
# Set background color to white
matplotlib.rc('axes', facecolor = 'white')

In [3]:
# os.chdir('..')
os.chdir('Documents/repos/geo')
# noinspection PyArgumentList
os.listdir()

['.git',
 '.gitattributes',
 '.gitignore',
 '.idea',
 '.ipynb_checkpoints',
 'data',
 'img',
 'notebooks',
 'presentations',
 'README.md',
 'src',
 '__pycache__']

## Load cleaned Teranet dataset with `da_id` column
Columns `da_id`, `da_city`, `da_median_tot_inc` were produced via a spatial join between Teranet points and DA polygons.

(for details, see notebook `Teranet_GTHA_DA_spatial_join.ipynb`)

In [4]:
# column `pin` will be converted to dtype=category
# after records with NaN `consideration_amt` will be dropped
dtypes = {
    'decade': 'int',
    'year': 'int',
    'lro_num': 'category',
    'postal_code': 'category',
    'street_designation': 'category',
    'street_direction': 'category',
    'municipality': 'category',
    'da_id': 'category',
    'da_city': 'category',
}
drop_cols = ['Unnamed: 0', 'index', 
             'index_right', 'province',
             'street_suffix']
t = time.time()
teranet_path = 'data/HHSaleHistory_cleaned_v0.9_GTHA_DA.csv'
df = pd.read_csv(teranet_path, 
                 parse_dates=['registration_date'],
                 dtype=dtypes)
df = df.drop(drop_cols, axis=1)
df = df.sort_values(['pin',
                     'registration_date'])
elapsed = time.time() - t
print("----- DataFrame with Teranet records loaded"
      "\nin {0:.2f} seconds".format(elapsed) + 
      "\nwith {0:,} rows\nand {1:,} columns"
      .format(df.shape[0], df.shape[1]) + 
      "\n-- Column names:\n", df.columns)

----- DataFrame with Teranet records loaded
in 34.69 seconds
with 6,062,853 rows
and 18 columns
-- Column names:
 Index(['registration_date', 'decade', 'year', 'lro_num', 'pin',
       'consideration_amt', 'postal_code', 'unitno', 'street_name',
       'street_designation', 'street_direction', 'municipality',
       'street_number', 'x', 'y', 'da_id', 'da_city', 'da_median_tot_inc'],
      dtype='object')


In [5]:
df.head()

,registration_date,decade,year,lro_num,pin,consideration_amt,postal_code,unitno,street_name,street_designation,street_direction,municipality,street_number,x,y,da_id,da_city,da_median_tot_inc
5242184,1986-07-09,198,1986,65,29000001,185000.0,L3P6K5,NaN,Cairns,Dr,NaN,Markham,120.0,-79.280389,43.887265,1608,Markham,28032.0
5242178,1986-04-14,198,1986,65,29000002,171000.0,L3P6K5,NaN,Cairns,Dr,NaN,Markham,118.0,-79.280309,43.887118,1608,Markham,28032.0
5242214,1988-09-09,198,1988,65,29000003,318000.0,L3P6K5,NaN,Cairns,Dr,NaN,Markham,116.0,-79.280255,43.887000,1608,Markham,28032.0
5242377,1999-01-29,199,1999,65,29000003,273000.0,L3P6K5,NaN,Cairns,Dr,NaN,Markham,116.0,-79.280255,43.887000,1608,Markham,28032.0
5242545,2011-02-18,201,2011,65,29000003,558000.0,L3P6K5,NaN,Cairns,Dr,NaN,Markham,116.0,-79.280255,43.887000,1608,Markham,28032.0


In [6]:
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6062853 entries, 5242184 to 6062631
Data columns (total 18 columns):
registration_date     6062853 non-null datetime64[ns]
decade                6062853 non-null int32
year                  6062853 non-null int32
lro_num               6062853 non-null category
pin                   6062853 non-null int64
consideration_amt     4637584 non-null float64
postal_code           5534039 non-null category
unitno                1339242 non-null float64
street_name           5874264 non-null object
street_designation    5516855 non-null category
street_direction      520896 non-null category
municipality          6058942 non-null category
street_number         5870282 non-null float64
x                     6062853 non-null float64
y                     6062853 non-null float64
da_id                 6062853 non-null category
da_city               6062853 non-null category
da_median_tot_inc     6021553 non-null float64
dtypes: category(7), datetime

### Drop all records with missing (or <$30) `consideration_amt` 

In [7]:
df = df.dropna(subset=['consideration_amt'])
df['pin'] = df['pin'].astype('category')
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4637584 entries, 5242184 to 6062631
Data columns (total 18 columns):
registration_date     4637584 non-null datetime64[ns]
decade                4637584 non-null int32
year                  4637584 non-null int32
lro_num               4637584 non-null category
pin                   4637584 non-null category
consideration_amt     4637584 non-null float64
postal_code           4342478 non-null category
unitno                1140626 non-null float64
street_name           4588439 non-null object
street_designation    4327098 non-null category
street_direction      424608 non-null category
municipality          4635981 non-null category
street_number         4587129 non-null float64
x                     4637584 non-null float64
y                     4637584 non-null float64
da_id                 4637584 non-null category
da_city               4637584 non-null category
da_median_tot_inc     4608506 non-null float64
dtypes: category(8), datet

### Unique records by `pin`
Column `pin` can be used for grouping operations with Teranet data.

In [8]:
df['pin'].describe()

count       4637584
unique      2219927
top       211080033
freq            464
Name: pin, dtype: int64

### Unique records by `xy`
Column `xy` is produced by concatenating `x` and `y` coordinates as strings. It can be used for grouping operations with Teranet data.

In [9]:
df['xy'] = df['x'].astype('str') + df['y'].astype('str')
df['xy'] = df['xy'].astype('category')
df['xy'].describe()

count                                 4637584
unique                                1394189
top       -79.413759934962343.776162782863395
freq                                     7222
Name: xy, dtype: object

There is a significant discrepancy between the number of records with unique `pin` and the number of records with unique `xy` (#  un`xy` < # with same `pin`).

## Add column `total_sales`
Total records for each pin, generated as a separate DataFrame `df_pin` which represents Teranet records grouped and indexed by `pin`.

`total_sales_pin` is added as a new column for Teranet records via a merge operation on `pin`.

In [10]:
# add column with total sales per `pin` and `xy` pair
print("Adding new columns...")
# group records by `pin`
t = time.time()
pin_counts = \
    df.groupby('pin')['consideration_amt'].count()
pin_counts.name = 'pin_total_sales'
df = pd.merge(df, pin_counts, on='pin')
elapsed = time.time() - t
print("\nNew column 'pin_total_sales' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

# group records by `xy` coordinate pairs
t = time.time()
xy_counts = \
    df.groupby('xy')['consideration_amt'].count()
xy_counts.name = 'xy_total_sales'
df = pd.merge(df, xy_counts, on='xy')
elapsed = time.time() - t
print("\nNew column 'xy_total_sales' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

Adding new columns...

New column 'pin_total_sales' added to the DataFrame!
took 3.16 seconds.

New column 'xy_total_sales' added to the DataFrame!
took 6.34 seconds.


## Add column `prev_sales`
New columns are added to Teranet records capturing, for each transaction, a rolling count of previous records from this `pin` or `xy` coordinate pair.

In [11]:
print("Adding new columns...")

df['count'] = 1 # used to produce rolling counts per `pin` and `xy`

# group by `pin`
t = time.time()
df['pin_prev_sales'] = \
    df.sort_values(['pin', 'registration_date'])\
    .groupby('pin')['count'].cumsum() - 1
elapsed = time.time() - t
print("\nNew column 'pin_prev_sales' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

# group by xy pairs
t = time.time()
df['xy_prev_sales'] = \
    df.sort_values(['xy', 'registration_date'])\
    .groupby('xy')['count'].cumsum() - 1
elapsed = time.time() - t
print("\nNew column 'xy_prev_sales' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

df = df.drop('count', axis=1)

Adding new columns...

New column 'pin_prev_sales' added to the DataFrame!
took 3.62 seconds.

New column 'xy_prev_sales' added to the DataFrame!
took 4.09 seconds.


## Add columns `price_cum_sum` and `price_pct_change`
New columns are added to Teranet records capturing, for each transaction, a rolling sum of price from previous records from this `pin` or `xy` coordinate pair, and `pct_change` compared to previous transaction from this `pin` or `xy` pair.

In [12]:
print("Adding new columns...")
# `price_cum_sum`
# group records by `pin`
t = time.time()
df['pin_price_cum_sum'] = \
    df.sort_values(['pin', 'registration_date'])\
    .groupby('pin')['consideration_amt'].cumsum()
elapsed = time.time() - t
print("\nNew column 'pin_price_cum_sum' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))
# group records by `xy` pairs
t = time.time()
df['xy_price_cum_sum'] = \
    df.sort_values(['xy', 'registration_date'])\
    .groupby('xy')['consideration_amt'].cumsum()
elapsed = time.time() - t
print("\nNew column 'xy_price_cum_sum' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

# `price_pct_change`
# group records by `pin`
t = time.time()
df['pin_price_pct_change'] = \
    df.sort_values(['pin', 'registration_date'])\
    .groupby('pin')['consideration_amt'].pct_change()
elapsed = time.time() - t
print("\nNew column 'pin_price_pct_change' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))
# group records by `xy`
t = time.time()
df['xy_price_pct_change'] = \
    df.sort_values(['xy', 'registration_date'])\
    .groupby('xy')['consideration_amt'].pct_change()
elapsed = time.time() - t
print("\nNew column 'xy_price_pct_change' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

Adding new columns...

New column 'pin_price_cum_sum' added to the DataFrame!
took 3.24 seconds.

New column 'xy_price_cum_sum' added to the DataFrame!
took 3.87 seconds.

New column 'pin_price_pct_change' added to the DataFrame!
took 4.28 seconds.

New column 'xy_price_pct_change' added to the DataFrame!
took 4.68 seconds.


## Add column `price_da_pct_change`
New column is added to Teranet records capturing, for each transaction, percentage change in price compared to the previous record from this `da_id`.

In [13]:
# add column 'price_da_pct_change' to Teranet records DataFrame 
t = time.time()
df['price_da_pct_change'] = \
    df.sort_values(['da_id', 'registration_date'])\
    .groupby('da_id')['consideration_amt'].pct_change()
elapsed = time.time() - t
print("\nNew column 'price_da_pct_change' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))


New column 'price_da_pct_change' added to the DataFrame!
took 3.86 seconds.


## Add columns `years_since_last_sale`
New columns are added to Teranet records capturing, for each transaction, years passed since the previous record from this `pin` or `xy` coordinate pair.

In [14]:
print("Adding the new column...")
# add column 'prev_visits' to Teranet records DataFrame 
t = time.time()
df['pin_years_since_last_sale'] = \
    df.sort_values(['pin', 'registration_date'])\
      .groupby('pin')['registration_date']\
        .diff().dt.days / 365
elapsed = time.time() - t
print("New column 'pin_years_since_last_sale' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

New column 'pin_years_since_last_sale'added to the DataFrame!
took 799.07 seconds.


In [15]:
print("Adding the new column...")
# add column 'prev_visits' to Teranet records DataFrame 
t = time.time()
df['xy_years_since_last_sale'] = \
    df.sort_values(['xy', 'registration_date'])\
      .groupby('xy')['registration_date']\
        .diff().dt.days / 365
elapsed = time.time() - t
print("New column 'xy_years_since_last_sale' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

Adding the new column...
New column 'xy_years_since_last_sale'added to the DataFrame!
took 495.81 seconds.


## Add columns `da_days_since_last_sale` and `da_years_since_last_sale`
New columns are added to Teranet records capturing, for each transaction, years passed since the previous record from this `pin` or `xy` coordinate pair.

In [24]:
print("Adding new columns...")
# add column 'da_days_since_last_sale' to Teranet records DataFrame 
t = time.time()
df['da_days_since_last_sale'] = \
    df.sort_values(['da_id', 'registration_date'])\
      .groupby('da_id')['registration_date']\
        .diff().dt.days
elapsed = time.time() - t
print("New column 'da_days_since_last_sale' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))
# add column 'da_years_since_last_sale' to Teranet records DataFrame 
t = time.time()
df['da_years_since_last_sale'] = \
    df.sort_values(['da_id', 'registration_date'])\
      .groupby('da_id')['registration_date']\
        .diff().dt.days / 365
elapsed = time.time() - t
print("New column 'da_years_since_last_sale' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

Adding new columns...
New column 'da_days_since_last_sale' added to the DataFrame!
took 9.16 seconds.
New column 'da_years_since_last_sale' added to the DataFrame!
took 7.06 seconds.


## Add columns `sale_next_6m/1y/3y` per `pin` and `xy`
New columns are added to Teranet records capturing, for each transaction, whether there would be another transaction in the future from this `pin`, `xy`, or `da_id`

Time horizons used: 6 months, 1 year, 3 years.

In [37]:
print("Adding new columns...")

# create a new column, marks True if next 'day_diff' <= 5

# group records by `pin`
t = time.time()
df = df.sort_values(['pin', 'registration_date'])
df['pin_sale_next_6m'] = \
    df['pin_years_since_last_sale'].shift(-1) <= 0.5
df['pin_sale_next_1y'] = \
    df['pin_years_since_last_sale'].shift(-1) <= 1
df['pin_sale_next_3y'] = \
    df['pin_years_since_last_sale'].shift(-1) <= 3
elapsed = time.time() - t
print("New columns 'pin_sale_next_..' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

# group records by `xy`
t = time.time()
df = df.sort_values(['xy', 'registration_date'])
df['xy_sale_next_6m'] = \
    df['xy_years_since_last_sale'].shift(-1) <= 0.5
df['xy_sale_next_1y'] = \
    df['xy_years_since_last_sale'].shift(-1) <= 1
df['xy_sale_next_3y'] = \
    df['xy_years_since_last_sale'].shift(-1) <= 3
elapsed = time.time() - t
print("New columns 'xy_sale_next_..' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

Adding new columns...
New columns 'pin_sale_next_..' added to the DataFrame!
took 3.61 seconds.
New columns 'xy_sale_next_..' added to the DataFrame!
took 3.11 seconds.


## Save results to a new .csv file
Teranet dataset without NaN records and with 12 new columns is saved as:
`data/HHSaleHistory_cleaned_v0.9_GTHA_DA_with_cols_v0.9.csv`

In [38]:
t = time.time()
save_path = 'data/HHSaleHistory_cleaned_v0.9_GTHA_DA_with_cols_v0.9.csv'
df.to_csv(save_path, index=False)
elapsed = time.time() - t
print("File saved to path:\n'" + save_path + "'" + 
      "\ntook {0:.2f} seconds.".format(elapsed))

File saved to path:
'data/HHSaleHistory_cleaned_v0.9_GTHA_DA_with_cols_v0.9.csv'
took 333.41 seconds.
